In [1]:
from keras.models import load_model
import json
import pickle

Using TensorFlow backend.


In [2]:
model = load_model('glove100d.hdf5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [3]:
with open("./data/input.json", "r") as fp:
    claim_text = json.load(fp)["text"]

In [4]:
preload_path = "data_dump_glove.data"
data = pickle.load(open(preload_path, "rb"))
data_train = data["X_train"]
labels_train = data["Y_train"]
data_test = data["X_test"]

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data_train, labels_train, test_size=0.2, random_state=101)
X_train_headlines = [i[0] for i in X_train]
X_train_articles = [i[1] for i in X_train]
X_test_headlines = [i[0] for i in X_test]
X_test_articles = [i[1] for i in X_test]

max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
# fit headline
tokenizer.fit_on_texts(X_train_headlines + X_train_articles)

In [7]:
# padding
max_words_headline = 70
max_words_article = 1000

# preprocess claim text
claim_text_seq = tokenizer.texts_to_sequences([claim_text])
claim_text_seq = sequence.pad_sequences(claim_text_seq, maxlen=max_words_headline)

In [8]:
with open("./results/er_opt.json", "r") as fp:
    df = json.load(fp)
    for i in range(len(df["articles"])):
        article = str(json.loads(df["articles"][i]["body"]))
        credible_text_seq = tokenizer.texts_to_sequences([article])
        credible_text_seq = sequence.pad_sequences(credible_text_seq, maxlen=max_words_article)
        print(model.predict([claim_text_seq, credible_text_seq]))

TypeError: string indices must be integers

In [11]:
with open("./results/er_opt.json", "r") as fp:
    df = json.load(fp)

credible_text_seq = tokenizer.texts_to_sequences([df["articles"][0]])
credible_text_seq = sequence.pad_sequences(credible_text_seq, maxlen=max_words_article)
res = model.predict([claim_text_seq, credible_text_seq])

In [12]:
res

array([[0.9908524]], dtype=float32)

In [14]:
type(res)

numpy.ndarray

In [17]:
res.tolist()[0][0]

0.990852415561676